In [2]:
import pandas as pd
from subprocess import call
import itertools
import numpy as np
from rpy2 import robjects
from rpy2.robjects import pandas2ri


In [3]:
ASE_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/ASE/nano_merge_ASE_SNP.csv"
m6A_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/m6A/nano_merge_m6A_SNP.csv"
pseU_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/pseU/nano_merge_pseU_SNP.csv"
stability_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/stability/nano_merge_stability_SNP.csv"
isoform_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/isoform/nano_merge_isoform_SNP.csv"
polyA_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/apa/nano_merge_polyA_SNP.csv"
promoter_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/promoter/nano_merge_promoter_SNP.csv"
tss_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/tss/nano_merge_tss_SNP.csv"

gene_bed_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/stability/gene_merge.bed"
outdir = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/colco"

chrom = "chr1"
strand = "+"
min_qtl_count=25
coloc_threshold = 0.75
p_threshold=0.05

In [4]:
def read_file(chrom, strand, full_summury, trait, trait_col=''):
    df = pd.read_csv(full_summury)
    df = df[(df['chrom'] == chrom)&(df['strand'] == strand)]
    df = df[df['p_value']<1]
    df = df.rename(columns={'p_value': f'p_value_{trait}'})
    df = df['snp_pos_1base'].astype(str)
    common_col = ['rsID', 'chrom', 'snp_pos_1base', 'strand', 'EAF', f'p_value_{trait}']
    if trait_col:
        common_col.append(trait_col)
    df = df[common_col]
    return df

In [7]:
## 读所有的文件
ASE_df = read_file(chrom, strand, ASE_file, "ASE")
m6A_df = read_file(chrom, strand, m6A_file, "m6A", "m6A_pos_1base")
pseU_df = read_file(chrom, strand, pseU_file, "pseU", "pseU_pos_1base")
stability_df = read_file(chrom, strand, stability_file, "stability")
isoform_df = read_file(chrom, strand, isoform_file, "isoform")
polyA_df = read_file(chrom, strand, polyA_file, "polyA")
promoter_df = read_file(chrom, strand, promoter_file, "promoter")
tss_df = read_file(chrom, strand, tss_file, "tss")

## 合并所有的 DataFrame
dfs = [ASE_df, stability_df, isoform_df, polyA_df, promoter_df, tss_df, m6A_df, pseU_df]
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on=['rsID', 'chrom', 'snp_pos_1base', 'strand', 'EAF'], how='outer')
merged_df['non_null_count'] = merged_df.notnull().sum(axis=1) # 统计每行的非空值数量
merged_df = merged_df[merged_df['non_null_count'] >= 10]  # 4列键 + 至少2种trait(6列)，保证这个SNP至少是两种QTL
merged_df = merged_df.drop(columns=['non_null_count'])

## gene注释
gene_df = merged_df[['chrom', 'snp_pos_1base', 'strand', 'rsID']]
gene_df = gene_df.drop_duplicates()
gene_df['snp_pos_0base'] = gene_df['snp_pos_1base'] - 1
gene_df['score'] = 0
gene_df = gene_df[['chrom', 'snp_pos_0base', 'snp_pos_1base', 'rsID', 'score', 'strand']]
gene_df.to_csv(f'{outdir}/rs_{chrom}_{strand}.bed', sep='\t', index=False, header=False)
call(f'bedtools intersect -wa -wb -s -a {outdir}/rs_{chrom}_{strand}.bed -b {gene_bed_file} > {outdir}/gene_rs_{chrom}_{strand}.bed',shell=True)
gene_df = pd.read_csv(f'{outdir}/gene_rs_{chrom}_{strand}.bed',header=None,sep='\t',usecols = [0,2,3,5,9],names=['chrom','snp_pos_1base','rsID','strand','geneID'])
merged_df = pd.merge(merged_df, gene_df[['chrom', 'snp_pos_1base', 'rsID', 'strand', 'geneID']], on=['chrom', 'snp_pos_1base', 'rsID', 'strand'], how='left')
call(f'rm {outdir}/rs_{chrom}_{strand}.bed {outdir}/gene_rs_{chrom}_{strand}.bed',shell=True)

## 分区进行colco
gene_groups = merged_df.groupby('geneID')
for gene_group in gene_groups:
    geneID = gene_group[0]
    gene_df = gene_group[1]
    genes_coloc_res = pd.DataFrame(columns=[
        "chrom","snp_pos_1base","rsID","strand","EAF",
        "p_value_1","p_value_2"
        "PPH4","trait1","trait2"
        "geneID"
        ])
    # 筛选1:首先对gene整体筛选 保证gene内至少有min_qtl_count个重叠QTL
    if len(set(gene_df['snp_pos_1base'])) >= min_qtl_count:
        gene_coloc_res = gene_2trait_combiner(geneID, gene_df, min_qtl_count, p_threshold, coloc_threshold)
        if gene_coloc_res:
            genes_coloc_res = pd.concat([genes_coloc_res, gene_coloc_res], ignore_index=True)
        else:
            continue
    else:
        print(f"{geneID} has less than {min_qtl_count} shared QTLs, skip")
        continue

if len(genes_coloc_res) != 0 :
    m6A1 = genes_coloc_res[genes_coloc_res['trait1'] == "m6A"]
    m6A2 = genes_coloc_res[genes_coloc_res['trait2'] == "m6A"]
    pseU1 = genes_coloc_res[genes_coloc_res['trait1'] == "pseU"]
    pseU2 = genes_coloc_res[genes_coloc_res['trait2'] == "pseU"]
    others = genes_coloc_res[
        (genes_coloc_res['trait1'] != "m6A") &
        (genes_coloc_res['trait1'] != "pseU") &
        (genes_coloc_res['trait2'] != "m6A") &
        (genes_coloc_res['trait2'] != "pseU")
    ]
    others = others.assign(m6A_pos_1base='', pseU_pos_1base='')

    m6A_df = m6A_df.rename(columns={'p_value_m6A': 'p_value_1'})
    m6A1 = pd.merge(
        m6A1,
        m6A_df[['chrom', 'snp_pos_1base', 'rsID', 'strand', 'EAF', 'm6A_pos_1base', 'p_value_1']],
        on=['chrom', 'snp_pos_1base', 'rsID', 'strand', 'EAF', 'p_value_1'],
        how='left'
    )
    m6A1['pseU_pos_1base'] = ''

    m6A_df = m6A_df.rename(columns={'p_value_1': 'p_value_2'})
    m6A2 = pd.merge(
        m6A2,
        m6A_df[['chrom', 'snp_pos_1base', 'rsID', 'strand', 'EAF', 'm6A_pos_1base', 'p_value_2']],
        on=['chrom', 'snp_pos_1base', 'rsID', 'strand', 'EAF', 'p_value_2'],
        how='left'
    )
    m6A2['pseU_pos_1base'] = ''


    pseU_df = m6A_df.rename(columns={'p_value_pseU': 'p_value_1'})
    pseU1 = pd.merge(
        pseU1,
        pseU_df[['chrom', 'snp_pos_1base', 'rsID', 'strand', 'EAF', 'pseU_pos_1base', 'p_value_1']],
        on=['chrom', 'snp_pos_1base', 'rsID', 'strand', 'EAF', 'p_value_1'],
        how='left'
    )
    pseU1['m6A_pos_1base'] = ''

    pseU_df = pseU_df.rename(columns={'p_value_1': 'p_value_2'})
    pseU2 = pd.merge(
        pseU2,
        pseU_df[['chrom', 'snp_pos_1base', 'rsID', 'strand', 'EAF', 'pseU_pos_1base', 'p_value_2']],
        on=['chrom', 'snp_pos_1base', 'rsID', 'strand', 'EAF', 'p_value_2'],
        how='left'
    )
    pseU2['m6A_pos_1base'] = ''

    result_df = pd.concat([m6A1, m6A2, pseU1, pseU2, others], ignore_index=True)
    result_df.to_csv(f'{outdir}/coloc_{chrom}_{strand}.csv', index=False)
else:
    print(f"{chrom} {strand} has no significant colocalization")

/tmp/ipykernel_31058/2141567306.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_summury)
/tmp/ipykernel_31058/2141567306.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_summury)


In [152]:
## 现在输入的是1个gene所有的share QTL，两两trait之间准备运行coloc
def gene_2trait_combiner(geneID, gene_df, min_qtl_count, p_threshold, coloc_threshold):
    trait_columns = ['tss', 'promoter', 'polyA', 'isoform', 'stability', 'm6A', 'pseU', 'ASE']
    for col in trait_columns:
        gene_df[col] = gene_df[f'p_value_{col}'].notna()
    ## 下面用来生成所有两两trait组合的df
    sub_dfs = {}
    for trait1, trait2 in itertools.combinations(trait_columns, 2):
        sub_df = gene_df[(gene_df[trait1] != False) & (gene_df[trait2] != False)]
        if len(set(sub_df['snp_pos_1base'])) >= min_qtl_count :
            sub_dfs[f"{trait1}_{trait2}"] = sub_df # 筛选2：两两trait组合的SNP数量大于阈值

    if len(sub_dfs) == 0:
        print(f'In {geneID},all trait combinations has less than {min_qtl_count} shared QTLs')
        return None
    else:
        gene_coloc_res = pd.DataFrame(columns=[
            "chrom","snp_pos_1base","rsID","strand","EAF",
            "p_value_1","p_value_2"
            "PPH4","trait1","trait2"
            ])
        for trait1_trait2, sub_df in sub_dfs.items():
            trait1, trait2 = trait1_trait2.split('_')
            sub_df = sub_df[["chrom" , "snp_pos_1base", "rsID", "strand", "EAF", f"p_value_{trait1}", f"p_value_{trait2}"]]
            sub_df = sub_df.drop_duplicates()
            if not {"m6A", "pseU"} & {trait1, trait2}:
                coloc_res = R_coloc_prepare(sub_df, trait1, trait2, p_threshold, False, coloc_threshold)
            else:
                coloc_res = R_coloc_prepare(sub_df, trait1, trait2, p_threshold, True, coloc_threshold)
            if coloc_res:
                gene_coloc_res = pd.concat([gene_coloc_res, coloc_res], ignore_index=True)
        if len(gene_coloc_res)==0:
            return None
        else:
            gene_coloc_res['geneID'] = geneID
            return gene_coloc_res # 返回chrom	snp_pos_1base	rsID	strand	EAF p_value_1	p_value_2 PPH4 trait1 trait2 geneID

In [ ]:
## 输入的是某两个trait之间的share QTL，处理一下重复的QTL，再运行colco
def R_coloc_prepare(df, trait1, trait2, p_threshold, have_mod, coloc_threshold):
    if (df[f"p_value_{trait1}"] > p_threshold).all() and (df[f"p_value_{trait2}"] > p_threshold).all():
        print(f"Both {trait1} and {trait2} are not significant in {geneID}")
        return None
    else:
        if have_mod:
            sub_df['idx'] = list(range(1,len(sub_df)+1))
            sub_df['snp_pos_1base'] = sub_df['snp_pos_1base'].astype(str)+"_"+sub_df['idx'].astype(str)
            sub_df['rsID'] = sub_df['rsID']+"_"+sub_df['idx'].astype(str)
            sub_df = sub_df.drop(columns=['idx'])
            new_column_names = {
                f"p_value_{trait1}": "p_value_1",
                f"p_value_{trait2}": "p_value_2"
            }
            sub_df = sub_df.rename(columns=new_column_names)
            coloc_res_df = R_coloc(sub_df, coloc_threshold)
        else:
            coloc_res_df = R_coloc(sub_df, coloc_threshold)
        if coloc_res_df:
            coloc_res_df['snp_pos_1base'] = coloc_res_df['snp_pos_1base'].apply(lambda x: x.split("_")[0])
            coloc_res_df['rsID'] = coloc_res_df['rsID'].apply(lambda x: x.split("_")[0])
            coloc_res_df['trait1'] = trait1
            coloc_res_df['trait2'] = trait2
            return coloc_res_df # 结果返回chrom	snp_pos_1base	rsID	strand	EAF p_value_1	p_value_2 PPH4 trait1 trait2


In [ ]:
def R_coloc(df, coloc_threshold):
    pandas2ri.activate()
    robjects.r('library(coloc)')
    r_df = pandas2ri.py2rpy(df) # 将 df 从 Pandas 数据框转换为 R 数据框，并赋值给 r_df
    robjects.globalenv['df'] = r_df # 将 r_df 传递给 R 环境，并在 R 环境中将其命名为 df
    robjects.globalenv['p1'] = p1
    robjects.globalenv['p2'] = p2
    robjects.globalenv['p12'] = p12
    r_code = """
    result <- coloc.abf(
        dataset1 = list(snp = df$rsID, pvalues = df$p_value_1, type = "quant", N = 104),
        dataset2 = list(snp = df$rsID, pvalues = df$p_value_2, type = "quant", N = 104),
        MAF = df$EAF,
        p1 = {p1},
        p2 = {p2},
        p12 = {p12}
    )
    """
    robjects.r(r_code)
    coloc_all_result = robjects.r['result']
    # coloc_summary = coloc_all_result.rx2('summary')
    coloc_results = coloc_all_result.rx2('results')
    # priors = coloc_result.rx2('priors')
    py_df = pd.DataFrame(robjects.conversion.rpy2py(coloc_results))
    coloc_res_df = py_df[py_df['SNP.PP.H4'] > coloc_threshold]
    if len(coloc_res_df) == 0:
        return None
    else:
        coloc_res_df = coloc_res_df.rename(columns={'snp': 'rsID', 'SNP.PP.H4': 'PPH4'})
        merged_df = pd.merge(df, coloc_res_df[['rsID', 'PPH4']], on='rsID', how='left')
        return merged_df # 返回的结果就是所有PPH4大于阈值的行，并且最后一列是PPH4值